In [1]:
from data import OptiverDataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
import os

In [2]:
class OptiverRNN(nn.Module):
    def __init__(self, hidden_size=128, layers=2, dropout=0.2):
        super().__init__()
        self.rnn = nn.RNN(input_size=14, 
                            hidden_size=hidden_size, 
                            num_layers=layers, 
                            batch_first=True, 
                            dropout=dropout
                         )
        self.linear = nn.Linear(hidden_size, 1)
    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.linear(x)
        return x

In [3]:
def collate_fn(data):
    return torch.stack([item[0] for item in data]), torch.stack([item[1] for item in data])

In [9]:
def train_rnn(hidden_size, layers, dropout, num_epochs=5): 
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = OptiverRNN(hidden_size=hidden_size, layers=layers, dropout=dropout)
    model = model.to(device)
    
    batch_size = 1
    
    train_dataset = OptiverDataset(split='train')
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    val_dataset = OptiverDataset(split='val')
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    loss_fn = nn.L1Loss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)
    
    maes = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}')
        model.train()
        progress = tqdm(total=len(train_dataloader), desc='Training')
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x).squeeze(dim=2)
            
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            progress.set_postfix({'loss': loss.item()})
            progress.update()
        progress.close()
                
        model.eval()
        loss = 0
        num_items = 0
        progress = tqdm(total=len(val_dataloader), desc='Validating')
        for x, y in val_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x).squeeze(dim=2)
            loss += torch.sum(torch.absolute(y_pred - y)).item()
            num_items += torch.numel(y_pred)
            progress.update()
        progress.close()
        mae = loss/num_items
        maes.append(mae)
        print(f'MAE = {mae}')
    return min(maes)


In [16]:
import pandas as pd
hidden_sizes = [64, 128, 256]
layers = [2, 4, 6, 8]
dropouts = [0.0, 0.1, 0.2, 0.3, 0.4]

hidden_size_df = pd.DataFrame(columns=['hidden_size', 'mae'])
layer_df = pd.DataFrame(columns=['num_layers', 'mae'])
dropout_df = pd.DataFrame(columns=['dropout', 'mae'])

for hidden_size in hidden_sizes:
    mae = train_rnn(hidden_size=hidden_size, layers=4, dropout=0)
    hidden_size_df = hidden_size_df.append({'hidden_size': hidden_size, 'mae': mae}, ignore_index=True)

for num_layers in layers:
    mae = train_rnn(hidden_size=128, layers=num_layers, dropout=0)
    layer_df = layer_df.append({'num_layers': num_layers, 'mae': mae}, ignore_index=True)

for dropout in dropouts:
    mae = train_rnn(hidden_size=128, layers=4, dropout=dropout)
    dropout_df = dropout_df.append({'dropout': dropout, 'mae': mae}, ignore_index=True)

dropout_df




Loading data:   0%|          | 0/139 [00:00<?, ?it/s]


Loading data:  17%|█▋        | 24/139 [00:00<00:00, 238.59it/s]


Loading data:  37%|███▋      | 52/139 [00:00<00:00, 262.06it/s]


Loading data:  58%|█████▊    | 81/139 [00:00<00:00, 273.96it/s]


Loading data:  79%|███████▉  | 110/139 [00:00<00:00, 276.67it/s]


Loading data: 100%|██████████| 139/139 [00:00<00:00, 271.13it/s]



Loading data: 100%|██████████| 20/20 [00:00<00:00, 228.99it/s]

Epoch 1


Training:   0%|          | 0/139 [00:00<?, ?it/s]

Validating:   0%|          | 0/20 [00:00<?, ?it/s]

MAE = 5.987296019785503
Epoch 2


...

MAE = 5.988209037807144
Epoch 5


Training:   0%|          | 0/139 [00:00<?, ?it/s]

Validating:   0%|          | 0/20 [00:00<?, ?it/s]

MAE = 5.992675612398419


,dropout,mae
0,0.0,5.969434
1,0.1,5.960738
2,0.2,5.968734
3,0.3,5.979508
4,0.4,5.984534


In [7]:
hidden_size_df

,hidden_size,mae
0,64.0,5.960039
1,128.0,5.955404
2,256.0,5.975110


In [9]:
layer_df

,num_layers,mae
0,2.0,5.963617
1,4.0,5.966758
2,6.0,5.982367
3,8.0,5.966689


In [ ]:
dropout_df

In [5]:
def test():
    model = OptiverRNN(hidden_size=128, layers=2, dropout=0.1)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    train_dataset = OptiverDataset(split='train_val')
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
    loss_fn = nn.L1Loss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)
    model.train()
    for i in range(5):
        progress = tqdm(total=len(train_dataloader), desc='Training')
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x).squeeze(dim=2)
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            progress.set_postfix({'loss': loss.item()})
            progress.update()
        progress.close()
        
    test_dataset = OptiverDataset(split='test')
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    model.eval()
    loss = 0
    num_items = 0
    progress = tqdm(total=len(test_dataloader), desc='Testing')
    for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x).squeeze(dim=2)
        loss += torch.sum(torch.absolute(y_pred - y)).item()
        num_items += torch.numel(y_pred)
        progress.update()
    progress.close()
    mae = loss/num_items
    return mae
mae = test()

Loading data: 100%|██████████| 159/159 [00:00<00:00, 276.81it/s]


Training:   0%|          | 0/159 [00:00<?, ?it/s]

Training:   0%|          | 0/159 [00:00<?, ?it/s]

Training:   0%|          | 0/159 [00:00<?, ?it/s]

Training:   0%|          | 0/159 [00:00<?, ?it/s]

Training:   0%|          | 0/159 [00:00<?, ?it/s]

Loading data: 100%|██████████| 40/40 [00:00<00:00, 255.54it/s]


Testing:   0%|          | 0/40 [00:00<?, ?it/s]

In [7]:
mae

5.643662756524154
